In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

from sklearn.model_selection import train_test_split

import optuna
from sklearn.model_selection import KFold

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./Tabnet_Raw_final.csv')

In [3]:
df.replace((np.inf, -np.inf), np.nan, inplace=True)
df.dropna(inplace=True)

In [9]:
train = df[df['회계년도'] <= 2017]
test = df[df['회계년도'] > 2017]

X_train = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1)
X_test = test.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1)

y_train = train['부실']
y_test = test['부실']

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

X = X_train.values
y = y_train.values

def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                    lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                    mask_type=mask_type, n_shared=n_shared,
                    scheduler_params=dict(mode="min",
                                        patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience 
                                        min_lr=1e-5,
                                        factor=0.5,),
                    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                    verbose=0,
                    ) #early stopping
    kf = KFold(n_splits=3, random_state=42, shuffle=True)
    CV_score_array    =[]
    for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        clf = TabNetClassifier(**tabnet_params)
        clf.fit(X_train=X_train, y_train=y_train,
                eval_set=[(X_valid, y_valid)],
                patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                eval_metric=['auc'])
        CV_score_array.append(clf.best_cost)
    avg = np.mean(CV_score_array)
    return avg

In [10]:
study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
study.optimize(Objective, n_trials=20) # timeout=6*60

[I 2023-04-28 21:45:15,199] A new study created in memory with name: TabNet optimization


Stop training because you reached max_epochs = 72 with best_epoch = 71 and best_val_0_auc = 0.81428


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 72 with best_epoch = 67 and best_val_0_auc = 0.85482


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 39 with best_epoch = 24 and best_val_0_auc = 0.75725


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 19 with best_epoch = 4 and best_val_0_auc = 0.82934


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-04-28 22:33:22,683] Trial 19 finished with value: 0.8181553505376636 and parameters: {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 0.00019042501568447304, 'patienceScheduler': 10, 'patience': 15, 'epochs': 88}. Best is trial 10 with value: 0.8597263305499867.


In [11]:
TabNet_params = study.best_params

In [12]:
print(TabNet_params)

{'mask_type': 'entmax', 'n_da': 56, 'n_steps': 2, 'gamma': 1.4, 'n_shared': 3, 'lambda_sparse': 0.0001021180196306919, 'patienceScheduler': 4, 'patience': 23, 'epochs': 100}


In [13]:
final_params = dict(n_d=TabNet_params['n_da'], n_a=TabNet_params['n_da'], n_steps=TabNet_params['n_steps'], gamma=TabNet_params['gamma'],
                    lambda_sparse=TabNet_params['lambda_sparse'], optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                    mask_type=TabNet_params['mask_type'], n_shared=TabNet_params['n_shared'],
                    scheduler_params=dict(mode="min",
                                        patience=TabNet_params['patienceScheduler'],
                                        min_lr=1e-5,
                                        factor=0.5,),
                    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                    verbose=0,
                    )
epochs = TabNet_params['epochs']

In [14]:
clf = TabNetClassifier(**final_params)
clf.fit(X_train=X, y_train=y,
          patience=TabNet_params['patience'], max_epochs=epochs,
          eval_metric=['auc'])

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [15]:
# train
X_train_re = X_train.values
dt1_pred_train = clf.predict(X_train_re)

# test
X_test_re = X_test.values
dt1_pred = clf.predict(X_test_re)

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

rdict={'model':[], "acc_train":[], "auc_train":[], 'acc_test':[],'precision':[],'recall':[],'f1_score':[], 'AUC_test':[]}

results_train  = (round(accuracy_score(y_train,dt1_pred_train),2),round(precision_score(y_train,dt1_pred_train),2), round(recall_score(y_train,dt1_pred_train),2), round(roc_auc_score(y_train,dt1_pred_train),2))

results = (round(accuracy_score(y_test,dt1_pred),2),
                round(precision_score(y_test,dt1_pred),2),
                round(recall_score(y_test,dt1_pred),2),
                round(f1_score(y_test,dt1_pred),2),
                round(roc_auc_score(y_test,dt1_pred),2))

rdict['model'].append(clf)
rdict['acc_train'].append(results_train[0])
rdict['auc_train'].append(results_train[1])
rdict['acc_test'].append(results[0])
rdict['precision'].append(results[1])
rdict['recall'].append(results[2])
rdict['f1_score'].append(results[3])
rdict['AUC_test'].append(results[4])

print(results_train)
rdf_tabnet = pd.DataFrame(data=rdict)
rdf_tabnet

(0.97, 0.84, 0.38, 0.69)


,model,acc_train,auc_train,acc_test,precision,recall,f1_score,AUC_test
0,"TabNetClassifier(n_d=56, n_a=56, n_steps=2, ga...",0.97,0.84,0.96,0.33,0.14,0.2,0.56
